In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import cftime
import warnings
xr.set_options(keep_attrs=True)
warnings.filterwarnings('ignore')
import sys
sys.path.insert(1, '../')
import my_utils as myf
import importlib
importlib.reload(myf)

<module 'my_utils' from '/gpfsm/dhome/laroach1/analysis/analysis_modelE-so-sst_pub/processing/../my_utils.py'>

In [2]:
c6dir = myf.cmip_dir+'CMIP/NASA-GISS/GISS-E2-1-G/historical/'
sspdir = myf.cmip_dir+'ScenarioMIP/NASA-GISS/GISS-E2-1-G/ssp245/'

In [3]:
def open_cmip(ens):
    
    ds = xr.open_mfdataset(c6dir+ens+'/Amon/psl/gn/*/psl_Amon_GISS-E2-1-G_historical_'+ens+'_*.nc')
    
    filestr = 'historical'
    if os.path.isdir(sspdir+ens):
        print('extend with SSP245')
        ds2 = xr.open_mfdataset(sspdir+ens+'/Amon/psl/gn/*/psl_Amon_GISS-E2-1-G_ssp245_'+ens+'_*.nc')
        ds = xr.concat([ds,ds2],dim='time').sel(time=slice('1990','2021'))
        filestr = 'hist+ssp245'
    ds['time'] = ds.indexes['time'].to_datetimeindex()
    ds['names'] = ens
    ds = ds.set_coords('names')
    ds = ds.sel(time=slice('1990','2021'))
    ds = ds[['psl']]
    ds = ds.mean(dim='lon')

    return ds, filestr

In [4]:
def compute_sam(ds, outfile):
          
    y1 = str(ds.time.values[0])[:4]
    y2 = str(ds.time.values[-1])[:4]
    
    ds_clim = ds.groupby('time.month').mean('time')
    ds_anom = ds.groupby('time.month') -  ds_clim
    
    a1 = ds_anom.interp(lat=-40).psl
    a2 = ds_anom.interp(lat=-65).psl
    index = a1/a1.std(dim='time') - a2/a2.std(dim='time')
    ds['SAM'] = index
    ds = ds[['SAM']]
    ds.SAM.attrs['units'] = '-'
    ds.SAM.attrs['long_name'] = 'Southern Annular Mode index'
    ds.SAM.attrs['desc'] = 'Not detrended'
    if 'month' in ds:
        ds = ds.drop('month')
    ds.to_netcdf(myf.processed_dir+'/timeseries/sam/'+outfile+str(y1)+'-'+str(y2)+'.nc')
    
    return ds

In [5]:
runmap = pd.read_csv('../e2.1runmap.csv',sep=',')
myens = runmap['names'].values

In [6]:
for ens in myens:
    ds, filestr = open_cmip(ens)
    ds = compute_sam(ds,'sam_Amon_GISS-E2-1-G_'+filestr+'_'+ens+'_gn_')

extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245


In [7]:
import glob
merradir = '/discover/nobackup/projects/gmao/merra2/data/products/'
myfiles = sorted(glob.glob(merradir+'MERRA2_all/*/*/*instM*3d_asm_Np*.nc4'))
listds = []
for f in myfiles:
    ds = xr.open_dataset(f)[['SLP']].interp(lat=[-40,-65]).mean(dim='lon')
    listds.append(ds)
ds = xr.concat(listds,dim='time').sel(time=slice('1990','2021'))
ds['SLP'] = ds.SLP/100.-1000. # convert units
ds = ds.rename({'SLP':'psl'})
ds.psl.attrs['units'] = 'mb-1000'
ds['names'] = 'OBS-MERRA2'
ds = ds.set_coords('names')
ds = compute_sam(ds,'sam_Amon_OBS-MERRA2_')